In [12]:
from IPython.display import display, HTML
import pandas as pd
import numpy

In [13]:
text = open("../Lovers het list cleaned.csv").read()

In [14]:
shiplist_raw = [line for line in text.split("\n")[1:] if "/" in line]

In [15]:
from collections import Counter, defaultdict

In [16]:
shiplist = sorted(Counter(shiplist_raw).items(), key=lambda x:x[1], reverse=True)

In [17]:
char_in_pair_freq = defaultdict(int)
for pair, score in shiplist:
    try: 
        left_name, right_name = pair.split("/")
    except:
        print(f"špatný počet lomítek v {pair}")
    char_in_pair_freq[left_name] += score
    char_in_pair_freq[right_name] += score
sorted_char_in_pairs = sorted(Counter(char_in_pair_freq).items(), key=lambda x:x[1], reverse=True)

In [18]:
print("Nechme si jen jména s frekvencí větší než 100, zbyde nám jmen:", len([(char, freq) for char, freq in sorted_char_in_pairs if freq >= 100]))


Nechme si jen jména s frekvencí větší než 100, zbyde nám jmen: 34


In [19]:
filtered_char_in_ship_34_best = [(char, freq) for char, freq in sorted_char_in_pairs if freq >= 49]

In [20]:
empty = numpy.zeros((len(filtered_char_in_ship_34_best), len(filtered_char_in_ship_34_best)), dtype=int)

In [21]:
filtered_shiplist = []
filtered_char_without_freq = [ship for ship, _ in filtered_char_in_ship_34_best]

def index(name):
    """Harry je na nulté pozici, když mu zadám Harry, vrátí 0."""
    return filtered_char_without_freq.index(name)

for ship, count in shiplist:
    left_name, right_name = ship.split("/")
    if left_name in filtered_char_without_freq and right_name in filtered_char_without_freq:
        filtered_shiplist.append((ship, count))
        if left_name == right_name:
            print("Sebevztah", left_name, count)
            empty[index(left_name), index(right_name)] = count
        else:
            empty[index(left_name), index(right_name)] = count
            empty[index(right_name), index(left_name)] = count
        
        
len(filtered_shiplist)

Sebevztah Original_Character 9


330

In [22]:
from pandas import DataFrame

In [23]:
df = DataFrame(empty)

keep = numpy.triu(num.ones(df.shape)).astype('bool').reshape(df.size)


NameError: name 'num' is not defined

In [24]:
body = []
beautify_chars = [name.replace("_", "&nbsp;") for name in filtered_char_without_freq]
for row_number, name in enumerate(beautify_chars):
    start = f"<tr>"
    start += f"<td>{name}</td>"
    for col_number in range(len(beautify_chars)):
        value = empty[row_number, col_number]
        if col_number >= row_number:
            # obarvení podle hodnoty
            if value >= 500:
                start += f"<td style='background-color: #caffc4; width: 4ex'>{value}</td>"
            elif value >= 100:
                start += f"<td style='background-color: #c2e3bf; width: 4ex'>{value}</td>"
            elif value == 0:
                start += f"<td style='background-color: white; width: 4ex'>{value}</td>"
            else:
                start += f"<td style='background-color: #829487; width: 4ex'>{value}</td>"
        else:
            start += "<td></td>"
    start += "</tr>\n"
    body.append(start)

html_table = f"""
<table>
    <thead style="writing-mode: vertical-lr; text-orientation: use-glyph-orientation; text-align: right">
        <th></th><th>{"</th><th>".join(beautify_chars)}</th>
    </thead>
    {"".join(body)}
</table>
"""


In [25]:
HTML(html_table)

Hermione Granger,0,2395,644,770,0,26,0,0,4,333,416,147,0,0,70,0,120,1,181,0,51,80,43,0,0,83,38,4,0,0,30,2,30,1,0,0,19,0,70,5,0,0,0,0,0,19,0,26,6,0,23,0,6,8
Draco Malfoy,,0,54,0,0,0,142,108,0,0,0,0,79,0,0,71,0,42,0,2,0,0,0,2,5,0,0,2,116,2,0,8,0,3,1,4,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Harry Potter,,,0,22,9,1,644,38,1,65,9,3,9,0,2,90,0,81,3,13,30,0,2,4,6,1,5,0,4,2,2,9,3,58,0,8,1,15,0,1,0,0,0,0,2,1,3,11,0,1,4,0,3,0
Severus Snape,,,,0,147,1,12,148,0,0,0,4,58,0,1,7,0,1,0,7,1,0,0,3,18,0,0,0,0,0,0,10,0,1,26,3,0,1,0,0,0,0,0,0,0,1,0,0,0,4,0,0,0,0
Lily Evans Potter,,,,,0,836,0,0,0,4,0,20,0,0,8,0,7,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,7,0,0,0,0,0,0,1,0
James Potter,,,,,,0,1,15,0,0,0,3,6,0,0,0,0,0,0,0,0,0,0,2,3,0,0,0,0,0,0,4,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0
Ginny Weasley,,,,,,,0,0,0,14,11,9,0,0,2,1,8,0,0,0,0,0,16,0,0,7,0,1,0,0,1,0,31,0,0,0,2,0,1,0,0,0,0,0,0,0,0,2,2,0,0,0,0,0
Original Female Character,,,,,,,,0,14,56,7,92,0,0,49,0,12,0,41,0,4,30,9,0,0,2,19,2,0,0,14,0,5,0,0,0,10,0,4,5,18,0,1,0,0,14,0,0,0,0,10,4,13,1
Newt Scamander,,,,,,,,,0,0,0,0,39,478,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4,0,0,0,0,0,0,0,0,1,5,11,20,0,0,0,0,0,0,0,1,0,0
Tom Riddle,,,,,,,,,,0,0,0,9,0,0,8,0,3,0,0,1,0,0,5,1,0,0,0,0,0,0,1,0,1,5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Ron Weasley,,,,,,,,,,,0,0,2,0,0,7,0,31,0,1,0,0,0,0,0,0,0,0,5,2,0,0,0,3,0,1,1,1,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0


In [26]:
frame = pd.DataFrame(data=empty, index=filtered_char_without_freq, columns=filtered_char_without_freq)

In [42]:
frame

,Harry_Potter,Draco_Malfoy,Hermione_Granger,Sirius_Black,Severus_Snape,Remus_Lupin,Newt_Scamander,James_Potter,Ginny_Weasley,Tom_Riddle,...,Abraxas_Malfoy,Rubeus_Hagrid,Nagini,Dean_Winchester,Crowley_(Good_Omens),Gellert_Grindelwald_pretending_Percival_Graves,Padma_Patil,Aziraphale,Alice_Longbottom,Percy_Jackson
Harry_Potter,2,1,891,199,1717,50,1,26,1082,1185,...,1,13,2,11,1,0,2,0,0,10
Draco_Malfoy,1,0,2772,9,65,5,0,4,153,13,...,0,1,0,0,0,0,0,0,0,2
Hermione_Granger,891,2772,2,187,922,91,5,32,85,374,...,7,4,0,14,1,0,11,0,0,1
Sirius_Black,199,9,187,0,143,3343,0,278,9,2,...,0,1,0,0,0,0,0,0,2,0
Severus_Snape,1717,65,922,143,0,191,0,106,14,23,...,0,11,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Gellert_Grindelwald_pretending_Percival_Graves,0,0,0,0,0,0,3,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Padma_Patil,2,0,11,0,0,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
Aziraphale,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,55,0,0,0,0,0
Alice_Longbottom,0,0,0,2,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [27]:
frame.to_csv("lovers_matrix_het.csv")